In [3]:
#Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import folium
from geopy.geocoders import Nominatim
import time
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier  # Ejemplo de modelo
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer



In [4]:
pp = pd.read_csv('C:\\Users\\jvazquez1\\Desktop\\proyecto_data\\Ingenias_proyecto\\datos\\pp.csv', sep=';', encoding='utf-8', low_memory=False)

In [5]:
# Leemos nuevamente los datos
pp.head()

,Row type,Advertiser,Advertiser code,Agent,Agent code,Invoice number,Product,Month,Year,Recipient,...,Exchange rate,Invoice amount gross home currency,Invoice amount net home currency,Contract amount home currency,Deal value home currency,Invoice ID,Deal Status,Invoice Type,Invoice Status,Discount amount
0,Contract transmission,S.A. La Nacion,A08009,ZenithOptimedia Group Argentina S.A,B02803,2019DC000780,Opcionales,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,251.44,120.75,156,11693.5,1708.0,Approved,Debit,Exported,0
1,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000683,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,131.46,79.14,82,12365.66,1611.0,Approved,Debit,Exported,0
2,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000684,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,57.2,34.42,35,12365.66,1612.0,Approved,Debit,Exported,0
3,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000685,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,39.9,24.01,25,12365.66,1613.0,Approved,Debit,Exported,0
4,Contract transmission,Arcor S.A.I.C.,A07784,ZenithOptimedia Group Argentina S.A,B02803,2019DC000686,Mogul,9,2019,ZenithOptimedia Group Argentina S.A,...,5.986.232,146.46,88.23,91,12365.66,1614.0,Approved,Debit,Exported,0


In [6]:
#Antes que nada, chequeemos nuevamente que eliminamos todos los valores faltantes.
pp.isnull().sum()

Row type                                  0
Advertiser                                0
Advertiser code                           0
Agent                                     0
Agent code                                0
Invoice number                          947
Product                                   0
Month                                     0
Year                                      0
Recipient                                 0
Contract                                  0
Invoice amount gross                      0
 Commission amount                        0
Invoice amount net                        0
Channel                                   0
Channel code                              0
Item start date                           0
Item end date                             0
SAP code                                  0
Contract status                        3723
Invoice amount total                      0
Invoice date                            947
Invoice batch number            

¿Qué tipo de problemas estamos tratando de resolver? 
Desarrollar un modelo predictivo que permita identificar: 
*patrones de gasto publicitario, 
*tendencias estacionales 
*efectividad de los canales de publicidad 
en función de las variables disponibles en el dataset.

Variables:
Invoice amount gross= cantidad de dinero gastado en publicidad
Channel= canales que más se usan
Month y year = fechas que más se publicita

REGRESIÓN

In [7]:
# Elimino aquellas variables que no quiero incluir en el modelo y las guardo en x. LIMPIEZA
x = pp.drop(['Row type', 'Advertiser','Agent','Product', 'Unnamed: 44','Unnamed: 45','Unnamed: 46','Advertiser code','Agent code', 'Invoice number', 'Recipient', 'Contract', 'Channel code', 'Item start date', 'Item end date', 'SAP code', 'Invoice amount total', 'Invoice date', 'Invoice batch number', 'Invoice currency', 'Deal', 'Deal currency', 'Deal value', 'Invoice holding company', 'Sales holding company', 'Billing scenario', 'Incentive discount value', 'Exchange rate', 'Invoice amount gross home currency', 'Invoice amount net home currency', 'Contract amount home currency', 'Deal value home currency', 'Invoice ID', 'Deal Status', 'Invoice Type', 'Invoice Status','Discount amount'], 
                  axis=1)

In [8]:
x.head()

,Month,Year,Invoice amount gross,Commission amount,Invoice amount net,Channel,Contract status,Rep,Incentive discount %
0,9,2019,9345,2116.64,7228.36,DLA Home & Health Argentina,Approved,Soledad,9.0
1,9,2019,4883.76,146.51,4737.25,DLA Discovery Argentina,Approved,Soledad,0.0
2,9,2019,2124.2,63.73,2060.47,DLA Animal Planet Argentina,Approved,Soledad,0.0
3,9,2019,1482,44.46,1437.54,DLA TLC Argentina,Approved,Soledad,0.0
4,9,2019,5445,163.35,5281.65,DLA Discovery ID Argentina,Approved,Soledad,0.0


In [9]:
lista_atributos = x.columns

In [29]:
nombres = ["Soledad", "Francisco", "Karina", "Rocio", "Leonardo", "Martin", "Barter", "Representante",
           "Daniel", "Maria", "Ezequiel", "José", "Marcela", "Nicolas", "Carina", "Eugenia"]

pp = pd.DataFrame({'Nombres': nombres})

# Suposiciones sobre el género de los nombres
genero = []

for nombre in nombres:
    if nombre in ["Karina", "Rocio", "Maria", "Carina", "Marcela", "Eugenia", "Soledad"]:
        genero.append("Mujer")
    elif nombre in ["Francisco", "Leonardo", "Martin", "Ezequiel", "Daniel", "José", "Nicolas"]:
        genero.append("Hombre")
    else:
        genero.append("Desconocido")

# Agregar la columna de género al DataFrame
pp['Género'] = genero

print(pp)


          Nombres       Género
0         Soledad        Mujer
1       Francisco       Hombre
2          Karina        Mujer
3           Rocio        Mujer
4        Leonardo       Hombre
5          Martin       Hombre
6          Barter  Desconocido
7   Representante  Desconocido
8          Daniel       Hombre
9           Maria        Mujer
10       Ezequiel       Hombre
11           José       Hombre
12        Marcela        Mujer
13        Nicolas       Hombre
14         Carina        Mujer
15        Eugenia        Mujer


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Crear un DataFrame con la columna 'Channel' que contiene los canales
data = {
    'Channel': [
        'DLA Home & Health Argentina',
        'DLA Discovery Argentina',
        'DLA Animal Planet Argentina',
        'DLA TLC Argentina',
        'DLA Discovery ID Argentina',
        'DLA Discovery Kids Argentina',
        'DLA Discovery Kids Brazil',
        'DLA Disc Turbo Latin',
        'DLA Food Network Latin',
        'DLA TLC Latin',
        'DLA Discovery ID Latin',
        'DLA Animal Planet Latin',
        'DLA Discovery Venezuela Chile',
        'DLA Home & Health Venezuela Chile',
        'DLA Discovery Mexico',
        'DLA Animal Planet Mexico',
        'DLA Discovery ID Mexico',
        'DLA TLC Mexico',
        'DLA Discovery Latin',
        'DLA Home & Health Latin',
        'DLA TLC Brazil',
        'DLA Home & Health Brazil',
        'DLA Discovery ID Brazil',
        'DLA Discovery Brazil',
        'DLA Animal Planet Brazil',
        'DLA Discovery Kids Latin',
        'Golf Channel Latin',
        'DLA HGTV Latin',
        'DLA Discovery Kids Chile',
        'DLA Home & Health Mexico'
    ]
}

pp = pd.DataFrame(data)

# Crear una instancia de LabelEncoder
encoder = LabelEncoder()

# Codificar la columna 'Channel' en formato numérico
pp['Channel_encoded'] = encoder.fit_transform(pp['Channel'])

# Asegurarse de seleccionar las características adecuadas y el objetivo (X y y)
X = pp.drop(['Channel'], axis=1)
y = pp['Channel_encoded']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y ajustar un modelo de regresión, por ejemplo, RandomForestRegressor
regresor = RandomForestRegressor()
regresor.fit(X_train, y_train)



RandomForestRegressor()

In [44]:
import pandas as pd

# Supongamos que 'pp' es tu DataFrame con los datos
# Crear una copia del DataFrame para evitar modificar el original
pp_encoded = pp.copy()

# Aplicar la codificación one-hot a la columna 'Contract status'
pp_encoded = pd.get_dummies(pp_encoded, columns=['Contract status'], prefix='Contract_status')

# Obtener un diccionario a partir de las columnas codificadas
contract_status_dict = pp_encoded.filter(like='Contract_status_').to_dict(orient='list')

# El diccionario contendrá las categorías como claves y una lista de 0s y 1s como valores
print(contract_status_dict)


{'Contract_status_encoded': [0, 1, 7, 4, 3, 8, 5, 2, 6], 'Contract_status_Approved': [True, False, False, False, False, False, False, False, False], 'Contract_status_Cancelled': [False, True, False, False, False, False, False, False, False], 'Contract_status_Completed': [False, False, False, False, False, False, False, True, False], 'Contract_status_In Process': [False, False, False, False, True, False, False, False, False], 'Contract_status_Missing Copy': [False, False, False, True, False, False, False, False, False], 'Contract_status_Rejected': [False, False, False, False, False, False, True, False, False], 'Contract_status_Revision Rejected': [False, False, False, False, False, False, False, False, True], 'Contract_status_Tiempo Oportuno Revision': [False, False, True, False, False, False, False, False, False], 'Contract_status_Ultima Hora Revision': [False, False, False, False, False, True, False, False, False]}


In [10]:
# Ahora selecciono las etiquetas y las guardo en y. OK

y = pp['Invoice amount gross']

In [11]:
x, y = np.array(x), np.array(y)

In [12]:
x

array([[9, 2019, '9345', ..., 'Approved', 'Soledad', 9.0],
       [9, 2019, '4883.76', ..., 'Approved', 'Soledad', 0.0],
       [9, 2019, '2124.2', ..., 'Approved', 'Soledad', 0.0],
       ...,
       [8, 2023, ' $ -   ', ..., 'Approved', 'Rocio', 0.0],
       [8, 2023, ' $ -   ', ..., 'Approved', 'Soledad', 0.0],
       [8, 2023, ' $ -   ', ..., 'Approved', 'Soledad', 0.0]],
      dtype=object)

In [13]:
y[:10]

array(['9345', '4883.76', '2124.2', '1482', '5445', '1294.28', '8519.6',
       '129600', '129600', '4416'], dtype=object)

Separando Train / Test

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=42)

In [34]:
x.shape

(34762, 9)

In [35]:
X_train.shape

(27809, 9)

In [36]:
X_test.shape

(6953, 9)

Random Forest

In [37]:
# Inicializo el modelo
regresor = RandomForestRegressor(criterion='absolute_error', random_state=42)

In [48]:
# Entreno el modelo
regresor.fit(X_train, y_train)

RandomForestRegressor()

In [209]:
regresor.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'absolute_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [49]:
# Entrenar el modelo con los datos de entrenamiento preprocesados
model.fit(X_train, y_train)

# Predigo los valores para el set de testeo usando el modelo ya entrenado
y_pred = model.predict(X_test)

NameError: name 'model' is not defined

In [166]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred)

NameError: name 'y_pred' is not defined

### Support Vector Machine 

In [211]:
from sklearn.svm import SVR

In [212]:
# Inicializo el modelo
regresor_svr = SVR(C=1.0, epsilon=0.2)

In [213]:
# Entreno el modelo
regresor_svr.fit(X_train, y_train)

ValueError: could not convert string to float: 'Renault Argentina'

In [214]:
regresor_svr.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.2,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [215]:
# Predigo los valores para el set de testeo
y_pred_svr = regresor_svr.predict(X_test)

NotFittedError: This SVR instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [216]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred_svr)

NameError: name 'y_pred_svr' is not defined